https://www.riksdagen.se/sv/dokument-lagar/dokument/motion/centerpartiets-budgetmotion-2022_H9024121

In [14]:
import pandas as pd
import requests

pd.options.mode.chained_assignment = None

In [89]:
multiplier = 1_000

In [46]:
search_url = 'https://data.riksdagen.se/dokumentlista/'
params = {
    'sok': 'utgiftsområde',
    'doktyp': 'mot',
    'rm': '2021/22',
    'parti': 'C',
    'sort': 'rel',
    'sortorder': 'desc',
    'utformat': 'json',
    'a': 's'}

resp = requests.get(search_url, params=params)
data = resp.json()
docs = pd.DataFrame(data['dokumentlista']['dokument'])
docs = docs[['titel', 'undertitel', 'rm', 'dokument_url_html']]

In [48]:
docs['Utgiftsområde'] = docs.titel.str.extract('Utgiftsområde (\d+)')
docs['Utgiftsområde'] = docs['Utgiftsområde'].astype(int)
docs = docs.sort_values('Utgiftsområde').reset_index(drop=True)

In [74]:
def find_matching_table(tables):
    cols = ['Anslag', 'Namn', 'Regeringens förslag', 'Avvikelse från regeringen']
    for table in tables:
        if table.columns.shape == (4,) and 'Avvikelse från regeringen' in table.columns:
            break
    return table

In [75]:
def fetch_table(url, area):
    url = f'http:{url}'
    tables = pd.read_html(url, encoding='utf8', header=0)
    cols = ['Anslag', 'Namn', 'Regeringens förslag', 'Avvikelse från regeringen']
    df = find_matching_table(tables)
    df.columns = cols
    df = df.dropna(how='all')
    df = df[~df.Anslag.str.startswith('Summa')]
    df['Utgiftsområde'] = area
    return df

In [77]:
tables = []

for _, row in docs.iterrows():
    table = fetch_table(row.dokument_url_html, area=row.Utgiftsområde)
    tables.append(table)

In [98]:
df = pd.concat(tables, sort=False)

In [99]:
df = df.dropna(how='all')

In [100]:
df = df.replace('±0', '0')
df['Regeringens förslag'] = df['Regeringens förslag'].str.replace('\s+', '', regex=True)
df['Avvikelse från regeringen'] = df['Avvikelse från regeringen'].str.replace('\s+', '', regex=True)
df['Regeringens förslag'] = df['Regeringens förslag'].str.replace('−', '-')
df['Avvikelse från regeringen'] = df['Avvikelse från regeringen'].str.replace('−', '-')

In [101]:
df['Regeringens förslag'] = df['Regeringens förslag'].astype(int) * multiplier
df['Avvikelse från regeringen'] = df['Avvikelse från regeringen'].astype(int) * multiplier

In [103]:
df.to_csv('../data/budgetmotion-2022-c.csv', index=False)